Librerías

In [34]:
import pandas as pd
import string
import numpy as np
import re


data = pd.read_excel('../downloads/emitidos_octubre.xlsx', header=0, skiprows=1)

Transformación de reporte afip DE VENTAS

In [37]:
#TIPO DE FACTURA
data['codigo_fc'] = [int(re.match(r'(\d+)', x).group(1)) for x in data['Tipo']]
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

data['tipo2_new'] = np.where(data['codigo_fc'].isin([1,6,11]), "Factura", "Credito")
data['tipo2_new'] = np.where(data['codigo_fc'] == 201, "Pyme_fc", data['tipo2_new'])
data['tipo2_new'] = np.where(data['codigo_fc'] == 203, "Pyme_nc", data['tipo2_new'])

data['tipo3_new'] = np.where(data['codigo_fc'].isin([1,3,201,203]), "A", "C")
data['tipo3_new'] = np.where(data['codigo_fc'].isin([6,8]), "B", data['tipo3_new'])

data['tipo3_new']

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Número Desde'].astype(int)

# Cliente
data['Denominación Receptor'] = data['Denominación Receptor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Cliente'] = data['Denominación Receptor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Receptor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])
data['TOTAL_10.5'] = round(data['NETO 10.5'] + data['IVA 10.5'],2)
data['TOTAL_21'] = round(data['NETO 21'] + data['IVA 21'],2)


# PONER 0 A TODAS LAS COLUMNAS QUE NO TIENEN ALGO
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%', 'TOTAL_NO_GRAVADO', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 0','NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)
data['TOTAL_21'] = data['TOTAL_21'].astype(str)
data['TOTAL_10.5'] = data['TOTAL_10.5'].astype(str)

# PONER EN NEGATIVO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    mask = (data['tipo2_new'] == 'Credito') | (data['tipo2_new'] == 'Pyme_nc')
    data.loc[mask, col] = -data.loc[mask, col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','tipo2_new','tipo3_new', 'Factura',
        'Cliente','CUIT', 'TOTAL_10.5', 'TOTAL_21',
        'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,3237,PILISAR S A,33709180849,379961.2,30681.22,7158.85
1,01/10/2025,Factura,A,3236,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,70149.06,885106.0
2,01/10/2025,Pyme_fc,A,213,PILISAR S A,33709180849,0.0,911434.46,11500025.0
3,01/10/2025,Pyme_fc,A,212,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,911434.46,11500025.0
4,01/10/2025,Pyme_nc,A,16,NEWSAN SOCIEDAD ANONIMA,30642617555,-0.0,-911434.46,-11500025.0
...,...,...,...,...,...,...,...,...,...
102,30/10/2025,Factura,A,3299,PILISAR S A,33709180849,793245.19,115328.15,334119.7
103,30/10/2025,Factura,A,3297,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,107476.12,1356080.0
104,30/10/2025,Pyme_fc,A,219,NEW PGA S R L,30615265450,4059025.97,2012977.95,57270.88
105,31/10/2025,Factura,A,3300,NEW PGA S R L,30615265450,0.0,267835.4,3379413.0


Apply data transformation function to afip

In [38]:
from data_transformation import transform_inbound_invoices
from data_transformation import transform_outbound_invoices

data_inbound = pd.read_excel('../downloads/recibidos_octubre.xlsx', header=0, skiprows=1)
data_outbound = pd.read_excel('../downloads/emitidos_octubre.xlsx', header=0, skiprows=1)

df = transform_inbound_invoices(data = data_inbound)


In [39]:
df

,Fecha,Factura,Proveedor,CUIT,NETO 10.5,IVA 10.5,NETO 21,IVA 21,TOTAL_NO_GRAVADO
0,01/10/2025,00009-00011663,INVERSIONES MAIPO S A,30699533412,0.0,0.0,96600.0,20286.0,0.00
1,01/10/2025,00008-00089201,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,474597.45
2,01/10/2025,00008-00089208,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,1161272.00
3,01/10/2025,00003-00017721,FIGUEROA EDGARDO DANIEL,20165212577,0.0,0.0,32931.36,6911.5199999999995,273455.16
4,01/10/2025,00008-00089213,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,140000.00
...,...,...,...,...,...,...,...,...,...
394,31/10/2025,00003-00000124,HERNANDEZ FRANCISCO GUSTAVO Y ROSSI,30707654348,-0.0,-0.0,-339917.36,-71382.65,0.00
395,31/10/2025,00008-00008218,CARNIVAL SRL,30676579938,-0.0,-0.0,-0.0,-0.0,2106948.35
396,31/10/2025,00008-00008219,CARNIVAL SRL,30676579938,-0.0,-0.0,-0.0,-0.0,1924116.35
397,31/10/2025,00008-00008220,CARNIVAL SRL,30676579938,-0.0,-0.0,-0.0,-0.0,2094700.95


In [40]:
df_2 = transform_outbound_invoices(data = data_outbound)

df_2

,Fecha,Factura,Cliente,CUIT,NETO 10.5,IVA 10.5,NETO 21,IVA 21,TOTAL_NO_GRAVADO
0,01/10/2025,00002-00003237,PILISAR S A,33709180849,343856.29,36104.91,25356.38,5324.84,7158.85
1,01/10/2025,00002-00003236,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,0.0,57974.43,12174.63,885106.00
2,01/10/2025,00002-00000213,PILISAR S A,33709180849,0.0,0.0,753251.62,158182.84,11500025.00
3,01/10/2025,00002-00000212,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,0.0,753251.62,158182.84,11500025.00
4,01/10/2025,00002-00000016,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,0.0,753251.62,158182.84,11500025.00
...,...,...,...,...,...,...,...,...,...
102,30/10/2025,00002-00003299,PILISAR S A,33709180849,717868.95,75376.24,95312.52,20015.63,334119.70
103,30/10/2025,00002-00003297,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,0.0,88823.24,18652.88,1356080.00
104,30/10/2025,00002-00000219,NEW PGA S R L,30615265450,3673326.67,385699.3,1663618.14,349359.81,57270.88
105,31/10/2025,00002-00003300,NEW PGA S R L,30615265450,0.0,0.0,221351.57,46483.83,3379413.00


Transform bookit report

FACTURAS DE VENTA

In [41]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

with open("../outbound.html", encoding="utf-8") as f:
    html = f.read()

# Parsear con BeautifulSoup (auto repara etiquetas mal cerradas)
soup = BeautifulSoup(html, "html.parser")

# Si no encuentra una tabla explícita, la creamos a mano
table = soup.find("table")
if not table:
    wrapped = f"<table>{soup}</table>"
    table = BeautifulSoup(wrapped, "html.parser").find("table")

# Convertir a DataFrame
df = pd.read_html(StringIO(str(table)))[0]

df


,linea,FECHA,TIPO,FACTURA,DK,CUIT,NETO 10.5,NETO 21,IVA 21,IVA 10.5,NO GRAVADO,TOTAL,Unnamed: 12
0,7223,01Oct25,A,3236,NEWSAN SA,30642617555,0.00,57974.45,12174.63,0.00,885106.00,955255.08,Eliminar
1,7224,01Oct25,A,3237,PILISAR SA,33709180849,343856.24,25356.36,5324.84,36104.91,7158.85,417801.20,Eliminar
2,7225,02Oct25,A,3238,PROCTER GAMBLE ARGENTINA S.R.L.,30615265450,501029.14,39246.69,8241.80,52608.06,45548.80,646674.49,Eliminar
3,7227,01Oct25,A,213,PILISAR SA,33709180849,0.00,753251.64,158182.84,0.00,11500025.00,12411459.48,Eliminar
4,7228,02Oct25,A,3239,PILISAR SA,33709180849,0.00,10419.55,2188.10,0.00,159077.00,171684.65,Eliminar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,7370,30Oct25,A,3297,NEWSAN SA,30642617555,0.00,88823.24,18652.88,0.00,1356080.00,1463556.12,Eliminar
86,7371,30Oct25,A,3298,NEWSAN SA,30642617555,850678.73,61570.00,12929.70,89321.27,0.00,1014499.70,Eliminar
87,7372,30Oct25,A,3299,PILISAR SA,33709180849,717868.95,95312.54,20015.63,75376.24,334119.70,1242693.06,Eliminar
88,7373,30Oct25,A,219,PROCTER GAMBLE ARGENTINA S.R.L.,30615265450,3673326.63,1663618.17,349359.81,385699.29,57270.88,6129274.78,Eliminar


FACTURAS DE COMPRA

In [42]:
with open("../inbound.html", encoding="utf-8") as f:
    html = f.read()

# Parsear con BeautifulSoup (auto repara etiquetas mal cerradas)
soup = BeautifulSoup(html, "html.parser")

# Si no encuentra una tabla explícita, la creamos a mano
table = soup.find("table")
if not table:
    wrapped = f"<table>{soup}</table>"
    table = BeautifulSoup(wrapped, "html.parser").find("table")

# Convertir a DataFrame
df = pd.read_html(StringIO(str(table)))[0]

df

,linea,FECHA,FECHA DE FACTURA,TIPO,FACTURA,Proveedor,CUIT,NETO 10.5,NETO 21,IVA 10.5,IVA 21,NO GRAVADO,TOTAL,Unnamed: 13
0,17161,01Oct2025,01Oct2025,A,00009-00011663,INVERSIONES MAIPO S A,30699533412,0.00,96600.00,0.00,20286.00,0.00,116886.00,Editar Eliminar
1,17162,01Oct2025,01Oct2025,A,00008-00089201,CARNIVAL SRL,30676579938,0.00,0.00,0.00,0.00,474597.45,474597.45,Editar Eliminar
2,17163,01Oct2025,01Oct2025,A,00008-00089208,CARNIVAL SRL,30676579938,0.00,0.00,0.00,0.00,1161272.00,1161272.00,Editar Eliminar
3,17164,01Oct2025,01Oct2025,A,00003-00017721,FIGUEROA EDGARDO DANIEL,20165212577,0.00,32931.36,0.00,6911.52,273455.16,313298.04,Editar Eliminar
4,17165,01Oct2025,01Oct2025,A,00008-00089213,CARNIVAL SRL,30676579938,0.00,0.00,0.00,0.00,140000.00,140000.00,Editar Eliminar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,17466,27Oct2025,27Oct2025,A,00093-00014823,SLOTS MACHINES SA,30605979048,0.00,270504.00,0.00,56805.84,5410.08,332719.92,Editar Eliminar
301,17467,27Oct2025,27Oct2025,A,00008-00093867,CARNIVAL SRL,30676579938,0.00,0.00,0.00,0.00,6907491.00,6907491.00,Editar Eliminar
302,17468,27Oct2025,27Oct2025,A,00008-00093910,CARNIVAL SRL,30676579938,720006.95,0.00,75600.73,0.00,14317.73,809925.41,Editar Eliminar
303,17469,27Oct2025,27Oct2025,A,00001-00000237,LB S R L,33717971219,0.00,66115.71,0.00,13884.30,0.00,80000.01,Editar Eliminar
